In [2]:
import os
import cv2
import pickle
import numpy as np
import insightface
from sklearn.preprocessing import normalize
from tqdm import tqdm
import time
import boto3
from io import BytesIO
from django.conf import settings




# Parámetros S3
S3_BUCKET = 'hackathon-facesiupb'
S3_PREFIX = 'faces/'
S3_KEY    = 'embeddings/embeddings.pkl'  # ruta dentro de S3

# Inicializar cliente S3 con credenciales en el código
s3 = boto3.client(
    's3',
    region_name            = settings.AWS_REGION,
    aws_access_key_id      = settings.AWS_ACCESS_KEY_ID,
    aws_secret_access_key  = settings.AWS_SECRET_ACCESS_KEY,
)
# Inicializas tu modelo ArcFace igual que antes...
model = insightface.app.FaceAnalysis(allowed_modules=['detection', 'recognition'])
model.prepare(ctx_id=0)

def load_images_and_extract_embeddings(image_folder):
    embeddings, labels = [], []
    for file_name in tqdm(os.listdir(image_folder), desc="Extrayendo embeddings"):
        img = cv2.imread(os.path.join(image_folder, file_name))
        if img is None: continue
        for face in model.get(img):
            if face.embedding is not None:
                embeddings.append(face.embedding)
                labels.append(os.path.splitext(file_name)[0])
    return normalize(np.array(embeddings)), labels

# 1) Extrae y normaliza
embeddings, labels = load_images_and_extract_embeddings('faces/')

# 2) Empaqueta y vuelca a pickle en memoria
buffer = BytesIO()
pickle.dump({'embeddings': embeddings, 'labels': labels}, buffer)
buffer.seek(0)

# 3) Sube a S3 directamente desde memoria
s3.upload_fileobj(buffer, Bucket=S3_BUCKET, Key=S3_KEY)
print(f"✔ Embeddings subidos a s3://{S3_BUCKET}/{S3_KEY}")


C:\Users\juane\Documents\ProyectosProgramacion\Waly\.venv\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\juane/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\juane/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\juane/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: C:\Users\juane/.insightface\models\buffalo_l\genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\juane/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


Extrayendo embeddings: 100%|██████████| 4531/4531 [08:59<00:00,  8.39it/s]


✔ Embeddings subidos a s3://hackathon-facesiupb/embeddings/embeddings.pkl


In [3]:
import onnxruntime
print("✅ ONNX Runtime version:", onnxruntime.__version__)


✅ ONNX Runtime version: 1.15.1
